# Models

* Test some models.
* Get a performance reference.
* Improve previous models with GridSearch and hyperparameter tweaking.

In [5]:
from xtalphases.data.preprocess import *
from xtalphases import __userpath__ as user_path

In [6]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
import pandas as pd

In [11]:
user_path = r'D:/USP/IC/Repositório/xtalphases'

## Data

Loading data from samples using ```read_csv```.

In [12]:
s3_filename = user_path + '/exploration/data/processed/sample4.csv'

In [15]:
s3_df = pd.read_csv(s3_filename, engine='python');

In [18]:
s3_df.shape

(1347872, 32)

### Train and Test

* Sampling strategy: get the same amount of structures of each crystal system.

#### Encoding crystal system

In [19]:
from sklearn.preprocessing import LabelEncoder

In [20]:
encoder = LabelEncoder()

In [21]:
cs_cat = s3_df['name_H-M_alt']

In [23]:
cs_cat_enc = encoder.fit_transform(cs_cat); cs_cat_enc

array([10, 10, 10, ...,  6,  6,  6])

In [24]:
encoder.classes_

array(['C 1 2 1', 'C 2 2 21', 'I 41', 'P 1', 'P 1 21 1', 'P 21 21 2',
       'P 21 21 21', 'P 31 2 1', 'P 32 2 1', 'P 41 21 2', 'P 43 21 2',
       'P 61 2 2', 'P 63 2 2', 'P 65 2 2', 'R 3 2 :H', 'R 3 :H'],
      dtype=object)

In [29]:
s3_df['cs_enc'] = cs_cat_enc

#### Splitting dataset into train and test

In [30]:
from sklearn.model_selection import StratifiedShuffleSplit

In [31]:
split = StratifiedShuffleSplit(n_splits=1, random_state=42, test_size=0.2)

In [32]:
for train_index, test_index in split.split(s3_df, s3_df['cs_enc']):
    train_set = s3_df.iloc[train_index]
    test_set = s3_df.iloc[test_index]

## Processing

In [33]:
crystal = train_set.drop('PHIMODEL', axis=1)

In [34]:
crystal_labels = train_set['PHIMODEL'].copy()

In [35]:
crystal.columns

Index(['Unnamed: 0', 'index_h', 'index_k', 'index_l', 'FOBS', 'SIGFOBS',
       'FMODEL', 'FOM', 'RESOL', 'pdbx_r_free_flag', 'crystal_system',
       'IT_number', 'name_H-M_alt', 'name_Hall', 'space_group_name_H-M',
       'space_group_name_Hall', 'Int_Tables_number', 'length_a', 'length_b',
       'length_c', 'angle_alpha', 'angle_beta', 'angle_gamma', 'volume',
       'PHI_ERROR', 'SYNCHROTRON', 'SOLV', 'WILSON', 'MATTHEWS', 'MW', 'ID',
       'cs_enc'],
      dtype='object')

### Droping Columns

Drop columns:
* Not available before structure solving (```FMODEL, FOM, PHI_ERROR```)
* Containing redudant information (preliminarly, because of our initial assumptions and computational resources).

In [36]:
crystal.columns

Index(['Unnamed: 0', 'index_h', 'index_k', 'index_l', 'FOBS', 'SIGFOBS',
       'FMODEL', 'FOM', 'RESOL', 'pdbx_r_free_flag', 'crystal_system',
       'IT_number', 'name_H-M_alt', 'name_Hall', 'space_group_name_H-M',
       'space_group_name_Hall', 'Int_Tables_number', 'length_a', 'length_b',
       'length_c', 'angle_alpha', 'angle_beta', 'angle_gamma', 'volume',
       'PHI_ERROR', 'SYNCHROTRON', 'SOLV', 'WILSON', 'MATTHEWS', 'MW', 'ID',
       'cs_enc'],
      dtype='object')

In [37]:
crystal.drop(['Unnamed: 0', 'FMODEL', 'FOM', 'pdbx_r_free_flag', 'cs_enc', 'WILSON', 'ID', 'IT_number', 
              'name_Hall', 'space_group_name_H-M', 'space_group_name_Hall', 'Int_Tables_number', 'PHI_ERROR'], axis=1, inplace=True)

In [38]:
crystal.drop(['MATTHEWS'], axis=1, inplace=True)

In [39]:
crystal.head(10)

,index_h,index_k,index_l,FOBS,SIGFOBS,RESOL,crystal_system,name_H-M_alt,length_a,length_b,length_c,angle_alpha,angle_beta,angle_gamma,volume,SYNCHROTRON,SOLV,MW
714089,-76,12,11,274.6150,69.434200,2.03690,monoclinic,C 1 2 1,157.363,156.438,88.417,90.0,110.01,90.0,2045214.666,Y,44.94,229239.27
304266,17,0,-13,1564.5300,46.756600,5.06602,trigonal,R 3 2 :H,100.920,100.920,386.690,90.0,90.00,120.0,3410735.516,Y,63.33,60067.85
1062119,30,14,69,45.1193,4.602020,1.24413,tetragonal,P 41 21 2,51.500,51.500,143.000,90.0,90.00,90.0,379271.750,Y,47.90,20372.33
64982,9,5,12,266.2420,3.227710,4.57986,trigonal,P 31 2 1,92.854,92.854,76.942,90.0,90.00,120.0,574506.980,Y,48.65,20786.47
453973,17,7,14,201.9410,1.176410,2.05087,monoclinic,P 1 21 1,61.670,32.402,62.033,90.0,109.82,90.0,116613.421,Y,45.78,25864.60
1142358,24,10,27,75.5419,22.302900,1.77397,orthorhombic,P 21 21 21,63.770,63.100,69.480,90.0,90.00,90.0,279579.678,N,58.94,23830.27
910483,13,32,5,163.9700,5.260540,1.73810,orthorhombic,P 21 21 21,41.580,66.580,105.050,90.0,90.00,90.0,290820.071,Y,43.42,33931.36
1318193,9,0,60,308.3290,5.701130,3.04153,orthorhombic,P 21 21 21,68.039,76.003,199.336,90.0,90.00,90.0,1030799.960,Y,58.27,89621.46
413615,4,3,64,35.4327,0.337118,2.06192,hexagonal,P 65 2 2,76.400,76.400,134.400,90.0,90.00,120.0,679386.034,Y,50.76,23217.16
419647,15,5,39,146.2150,1.639790,2.51225,hexagonal,P 65 2 2,76.400,76.400,134.400,90.0,90.00,120.0,679386.034,Y,50.76,23217.16


### Encoding

This should be formalized using ```sklearn``` functions. 

In [40]:
to_encode = ['SYNCHROTRON', 'crystal_system', 'name_H-M_alt']

In [41]:
crystal_1h =  pd.get_dummies(crystal, columns=to_encode)

## Training Preliminar Models

* Metric: RMSE of phases (perhaps not the best option to evaluate error in this case).

In [57]:
from sklearn.model_selection import cross_val_score

In [58]:
from sklearn.metrics import mean_squared_error

Transform training encoded dataset into a ```numpy``` array if needed.

In [59]:
crystal_1h_arr = crystal_1h.values

```display_scores``` is convenient function to display cross-validation results

In [60]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

```cross_validate``` allows for more control over the cross validation process, enabling to get:
* Cross validation scores
* Trained models over each k-fold CV.

### Linear Regression

In [61]:
from sklearn.linear_model import LinearRegression

In [62]:
lin_reg = LinearRegression(n_jobs=3)

In [63]:
lin_reg_score = cross_val_score(lin_reg, crystal_1h_arr, crystal_labels, 
                               scoring='neg_mean_squared_error', cv=5)

In [64]:
display_scores(np.sqrt(-lin_reg_score))

Scores: [104.78804713 104.72078026 104.65947066 104.71900169 104.7604574 ]
Mean: 104.72955142972505
Standard deviation: 0.04353497498956512


### Decision Tree Regressor

In [63]:
from sklearn.tree import DecisionTreeRegressor

In [64]:
tree_reg = DecisionTreeRegressor(max_depth=400, max_leaf_nodes=50, max_features=1.0)

In [68]:
tree_reg_score = cross_val_score(tree_reg, crystal_1h_arr, crystal_labels,
                                 scoring='neg_mean_squared_error', cv=4)

In [69]:
display_scores(np.sqrt(-tree_reg_score))

Scores: [104.20102072 104.05304763 104.07230694 103.71057404]
Mean: 104.00923733188866
Standard deviation: 0.18157487834416125


### Random Forest Regressor

In [107]:
from sklearn.ensemble import RandomForestRegressor

In [108]:
forest_reg = RandomForestRegressor(n_estimators=100, max_depth=200, max_leaf_nodes=50, n_jobs=-1)

In [109]:
forest_reg_score = cross_val_score(forest_reg, crystal_1h_arr, crystal_labels,
                                  scoring='neg_mean_squared_error', cv=2)

In [110]:
display_scores(np.sqrt(-forest_reg_score))

Scores: [103.9782631  104.03668001]
Mean: 104.00747155422698
Standard deviation: 0.029208451857165585


#### Testing the model without cross-validation

In [34]:
test_reg = RandomForestRegressor(n_estimators=20, n_jobs=-1)

In [35]:
test_reg.fit(crystal_1h_arr, crystal_labels)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [36]:
test_reg_pred = test_reg.predict(crystal_1h_arr)

In [37]:
test_mse = mean_squared_error(test_reg_pred, crystal_labels)

In [38]:
display_scores(np.sqrt(test_mse))

Scores: 42.89033484273397
Mean: 42.89033484273397
Standard deviation: 0.0


In [207]:
test_reg_pred_test = test_reg.predict(processing_pipeline.fit_transform(test_set))

In [209]:
np.sqrt(mean_squared_error(test_reg_pred_test, test_set['PHIMODEL'].copy()))

107.04765794173612

#### Feature Importances

In [39]:
def display_features_importances(tree_model, dataframe):
    paired_features = zip(tree_model.feature_importances_, dataframe.columns)
    return sorted(paired_features, reverse=True)

In [73]:
display_features_importances(test_reg, crystal_1h)

[(0.21806264103367198, 'FOBS'),
 (0.21662124601905944, 'RESOL'),
 (0.21292330467015344, 'SIGFOBS'),
 (0.11410991685570256, 'index_k'),
 (0.10896457215187516, 'index_l'),
 (0.10476116777306632, 'index_h'),
 (0.003307501439833692, 'MW'),
 (0.0032531879431497656, 'length_a'),
 (0.0029233635060326585, 'length_b'),
 (0.0029081022872981138, 'SOLV'),
 (0.002695644041276695, 'length_c'),
 (0.002524166458407795, 'volume'),
 (0.0010978195306000505, 'angle_alpha'),
 (0.0010077808479985971, 'angle_gamma'),
 (0.0009323264391158982, 'angle_beta'),
 (0.0006587253055716325, 'crystal_system_orthorhombic'),
 (0.0006420309464474813, 'name_H-M_alt_I 4'),
 (0.0005631551156035925, 'crystal_system_tetragonal'),
 (0.0003602955322485874, 'name_H-M_alt_P 21 21 21'),
 (0.00035747336348354526, 'name_H-M_alt_P 1'),
 (0.0003306355677988264, 'crystal_system_triclinic'),
 (0.0002508912538942365, 'name_H-M_alt_P 2 21 21'),
 (0.0002240439344222608, 'crystal_system_monoclinic'),
 (0.00021208398956460594, 'name_H-M_alt_P

In [74]:
rfr_feature_importances = test_reg.feature_importances_

## Improving Models

In [74]:
from sklearn.pipeline import Pipeline, FeatureUnion

In [75]:
from sklearn.base import BaseEstimator, TransformerMixin

### Reference Regressor

In [76]:
class ReferenceRegressor(BaseEstimator):
    
    def fit(self, X, y=None):
        pass
    
    def predict(self, y):
        return -180.0+360*np.random.rand(y.shape[0])

### Processing Pipeline

#### DataFrame Selector

In [77]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.attribute_names].values

#### Specifying Attributes

In [78]:
cat_attributes = ['crystal_system', 'name_H-M_alt']

In [79]:
num_attributes = ['FOBS', 'SIGFOBS',
       'RESOL', 'length_a', 'length_b', 'length_c', 'angle_alpha', 'angle_beta', 'angle_gamma',
       'volume', 'MW']

In [80]:
special_cat_attributes = ['index_h', 'index_k', 'index_l']

#### Partial Pipelines

In [81]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [82]:
from sklearn.preprocessing import OneHotEncoder

In [83]:
num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attributes)),
    ('scaler', StandardScaler()),
])

In [84]:
cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attributes)),
    ('cat_encoder', OneHotEncoder(sparse=False)),
])

In [85]:
special_cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(special_cat_attributes)),   
])

#### Single Processing Pipeline

In [86]:
processing_pipeline = FeatureUnion(transformer_list=[
    ('num_pipeline', num_pipeline),
    ('cat_pipeline', cat_pipeline),
    ('special_cat_pipeline', special_cat_pipeline),
])

### Training ```RFR```

In [87]:
new_full_pipeline = Pipeline([
    ('processing_pipeline', processing_pipeline),
    ('training_model', RandomForestRegressor(n_estimators=30, n_jobs=-1))
])

In [88]:
test_set.columns

Index(['Unnamed: 0', 'index_h', 'index_k', 'index_l', 'FOBS', 'SIGFOBS',
       'FMODEL', 'PHIMODEL', 'FOM', 'RESOL', 'pdbx_r_free_flag',
       'crystal_system', 'IT_number', 'name_H-M_alt', 'name_Hall',
       'space_group_name_H-M', 'space_group_name_Hall', 'Int_Tables_number',
       'length_a', 'length_b', 'length_c', 'angle_alpha', 'angle_beta',
       'angle_gamma', 'volume', 'PHI_ERROR', 'SYNCHROTRON', 'SOLV', 'WILSON',
       'MATTHEWS', 'MW', 'ID', 'cs_enc'],
      dtype='object')

In [89]:
new_rfr = new_full_pipeline.fit(crystal, crystal_labels)

In [90]:
new_rfr_pred_train = new_rfr.predict(test_set)

In [91]:
np.sqrt(mean_squared_error(new_rfr_pred_train, test_labels))

NameError: name 'test_labels' is not defined

### GridSearchCV 

* Using GridSearch on RandomForestRegressor

In [120]:
from sklearn.model_selection import GridSearchCV

In [121]:
from sklearn.ensemble import RandomForestRegressor

In [122]:
rfr = RandomForestRegressor()

In [123]:
param_grid = {'n_estimators':[10, 20, 30], 'max_features':[4, 15, 28]}            

In [131]:
rfr_cv = GridSearchCV(rfr, param_grid, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2, cv=3)

In [132]:
rfr_cv.fit(crystal_1h_arr, crystal_labels)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 20.2min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [10, 20, 30], 'max_features': [4, 15, 28]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=2)

In [164]:
from sklearn.externals import joblib

In [165]:
joblib.dump(rfr_cv, 'rfr_cv.pkl')

['rfr_cv.pkl']

In [188]:
np.sqrt(-rfr_cv.best_score_)

107.51007593832867

In [189]:
rfr_cv.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=28, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=30, n_jobs=None, oob_score=False,
           random_state=None, verbose=0, warm_start=False)

In [ ]:
param_grid = {
    {'n_estimators':[10, 100, 500],
    'max_depth':[10, 50, 100, 500],
     'max_leaf_nodes':[20, 50, 100]}
    {'bootstrap':[False],
    'max_depth':[10, 50, 100, 500]
    }    
}

In [ ]:
rfr_cv = GridSearchCV(rfr, param_grid, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2, cv=3)

### Applying Best Params

In [171]:
full_pipeline = Pipeline([
    ('processing', processing_pipeline),
    ('train_model', RandomForestRegressor(n_jobs=-1, **rfr_cv.best_params_))
])

In [158]:
rfr_cv.best_params_

{'max_features': 28, 'n_estimators': 30}

In [159]:
crystal.shape

(541187, 32)

In [160]:
rfr_bp = full_pipeline.fit(crystal, crystal_labels)

In [161]:
rfr_bp_pred = rfr_bp.predict(crystal)

In [162]:
np.sqrt(mean_squared_error(rfr_bp_pred, crystal_labels))

41.56996342883055

##### Using test set (forbidden!)

In [121]:
test_labels = test_set['PHIMODEL'].copy()

In [182]:
rfr_bp_test_pred = rfr_bp.predict(test_set)

In [183]:
np.sqrt(mean_squared_error(rfr_bp_test_pred, test_labels))

107.2738356529547

### Selecting K Best

In [96]:
def indices_of_top_k(arr, k):
    return np.sort(np.argpartition(np.array(arr), -k)[-k:])

In [97]:
class SelectKBest():
    
    def __init__(self, k, feature_importances):
        self.k = k
        self.feature_importances = feature_importances
        
    def fit(self, X, y=None):
        self.feature_indices_ = indices_of_top_k(self.feature_importances, self.k)
        return self
    
    def transform(self, X):
        return X[:, self.feature_indices_]

In [190]:
full_pipeline2 = Pipeline([
    ('processing', processing_pipeline),
    ('feature_selection', SelectKBest(7, rfr_feature_importances)),
    ('train_model', RandomForestRegressor(n_jobs=-1, n_estimators=30))
])

In [191]:
rfr_kbest = full_pipeline2.fit(crystal, crystal_labels)

In [192]:
rfr_kbest_pred = rfr_kbest.predict(crystal)

In [193]:
np.sqrt(mean_squared_error(rfr_kbest_pred, crystal_labels))

41.55188225950301

In [194]:
rfr_kbest_pred_test = rfr_kbest.predict(test_set)

In [195]:
np.sqrt(mean_squared_error(rfr_kbest_pred_test, test_labels))

107.16324970374899

#### Increasing ```n_estimators```

In [92]:
full_pipeline3 = Pipeline([
    ('processing', processing_pipeline),
    ('train_model', RandomForestRegressor(n_jobs=-1, max_features=1.0, n_estimators=100))
])

In [ ]:
rfr_inc_nest = full_pipeline3.fit(crystal, crystal_labels)

In [ ]:
rfr_inc_nest_pred = rfr_inc_nest.predict(test_set)

In [ ]:
np.sqrt(mean_squared_error(rfr_inc_nest.predict(crystal), crystal_labels))

In [ ]:
np.sqrt(mean_squared_error(rfr_inc_nest_pred, test_labels))

#### Using ```ReferenceRegressor```

In [77]:
rg = ReferenceRegressor()

In [78]:
np.sqrt(mean_squared_error(rg.predict(crystal), crystal_labels))

147.35577843785953

---
## Conclusions 

* ```RandomForestRegressor``` is the most performant model until now. Explore some preparation steps using the smallest dataset (```sample1.csv```).
* We have a badly overfitting model, with twice the error on the test set.


**Further Objectives**
* Understanding how statistical distributions are specified in *scipy*. Study it for ```RandomizedGridSearch```.
* Implement ideas of pipelines used along with grid search as show in [Evan Miller Notebook](https://www.kaggle.com/evanmiller/pipelines-gridsearch-awesome-ml-pipelines).

**Further Studies**
* Normalized structure factors ($E$) in direct methods. 
* Take care of subtle normalizations and interpretation of each crystallographic feature.